# Notebook to demonstrate how to run FEISTY with multiple submissions

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from glob import glob
from subprocess import check_call

import FEISTY_driver
import matplotlib.pyplot as plt
import yaml

import feisty

## Specify case, setup info and forcing files

In [3]:
# case = 'g.e22.TL319_t13.G1850ECOIAF_JRA_HR.4p2z.001'
# case_sname = 'TL319_t13.4p2z.001'
# dir_forcing = '/glade/scratch/kristenk/hi-res-feisty-inputs'
# nyears_per_submission = 1

case = 'g.e22.GOMIPECOIAF_JRA-1p4-2018.TL319_g17.4p2z.001branch'
case_sname = 'TL319_g17.4p2z.001'
dir_forcing = '/glade/work/mlevy/codes/feisty/input_files/1deg_companion_run'
nyears_per_submission = 1  # TODO: need better control here

run_config_file = f'feisty-config.{case_sname}.yml'
run_forcing_file = f'feisty-forcing.{case_sname}.yml'
run_settings_file = f'feisty-settings.{case_sname}.yml'

In [4]:
run_config_info = dict(
    run_name=f"feisty.{case_sname}",
    num_chunks=32,
    ignore_year_in_forcing=False,
    diagnostic_names=[],
    max_output_time_dim=365,
    method="euler",
    output_in_2D=True,
    computing_account="P93300670",
    resubmit=True,
    run_dir_root="/glade/scratch/{{env['USER']}}",
    forcing_rename={
        "HT": "bathymetry",
        "time": "forcing_time",
        "pocToFloor": "poc_flux_bottom",
        "mesozooC_zint_100m_2": "zooC",
        "mesozoo_loss_zint_100m": "zoo_mort",
        "TEMP_BOTTOM": "T_bottom",
        "TEMP_mean_100m": "T_pelagic",
    },
)
with open(run_config_file, 'w') as fid:
    yaml.dump(run_config_info, fid)

check_call(['cat', run_config_file]);

computing_account: P93300670
diagnostic_names: []
forcing_rename:
  HT: bathymetry
  TEMP_BOTTOM: T_bottom
  TEMP_mean_100m: T_pelagic
  mesozooC_zint_100m_2: zooC
  mesozoo_loss_zint_100m: zoo_mort
  pocToFloor: poc_flux_bottom
  time: forcing_time
ignore_year_in_forcing: false
max_output_time_dim: 365
method: euler
num_chunks: 32
output_in_2D: true
resubmit: true
run_dir_root: /glade/scratch/{{env['USER']}}
run_name: feisty.TL319_g17.4p2z.001


In [5]:
forcing_files = sorted(glob(f'{dir_forcing}/{case}.pop.h.*.nc'))
datestrs = list({f.split('.')[-2] for f in forcing_files})

required_vars = [
    'TEMP_BOTTOM',
    'TEMP_mean_100m',
    'mesozooC_zint_100m_2',
    'mesozoo_loss_zint_100m',
    'pocToFloor',
]

forcing_file_groups = {k: {} for k in datestrs}

for datestr in datestrs:
    file_list_i = [f for f in forcing_files if datestr in f]
    for v in required_vars:
        assert v in [f.split('.')[-3] for f in file_list_i]

    forcing_file_groups[datestr]['list_forcing_files'] = file_list_i
    forcing_file_groups[datestr]['nyears'] = nyears_per_submission
    forcing_file_groups[datestr]['start_date'] = f'{datestr[:4]}-01-01'

forcing_file_groups['keys'] = list(sorted(forcing_file_groups.keys()))

with open(run_forcing_file, 'w') as fid:
    yaml.dump(forcing_file_groups, fid)

check_call(['head', '-n', '20', run_forcing_file]);

195801-196712:
  list_forcing_files:
  - /glade/work/mlevy/codes/feisty/input_files/1deg_companion_run/g.e22.GOMIPECOIAF_JRA-1p4-2018.TL319_g17.4p2z.001branch.pop.h.TEMP_BOTTOM.195801-196712.nc
  - /glade/work/mlevy/codes/feisty/input_files/1deg_companion_run/g.e22.GOMIPECOIAF_JRA-1p4-2018.TL319_g17.4p2z.001branch.pop.h.TEMP_mean_100m.195801-196712.nc
  - /glade/work/mlevy/codes/feisty/input_files/1deg_companion_run/g.e22.GOMIPECOIAF_JRA-1p4-2018.TL319_g17.4p2z.001branch.pop.h.mesozooC_zint_100m_2.195801-196712.nc
  - /glade/work/mlevy/codes/feisty/input_files/1deg_companion_run/g.e22.GOMIPECOIAF_JRA-1p4-2018.TL319_g17.4p2z.001branch.pop.h.mesozoo_loss_zint_100m.195801-196712.nc
  - /glade/work/mlevy/codes/feisty/input_files/1deg_companion_run/g.e22.GOMIPECOIAF_JRA-1p4-2018.TL319_g17.4p2z.001branch.pop.h.pocToFloor.195801-196712.nc
  nyears: 1
  start_date: '1958-01-01'
keys:
- 195801-196712


In [6]:
datestrs

['195801-196712']

In [7]:
feisty_settings = feisty.settings.get_defaults()
with open(run_settings_file, 'w') as fid:
    yaml.dump(feisty_settings, fid)

check_call(['head', '-n', '20', run_settings_file]);

benthic_prey:
  defaults:
    benthic_efficiency: 0.075
    carrying_capacity: 80.0
  members:
  - name: benthic_prey
fish:
  defaults:
    a_consumption: 20.0
    a_encounter: 70.0
    a_metabolism: 4.0
    assim_efficiency: 0.7
    b_consumption: 0.25
    b_encounter: 0.2
    b_metabolism: 0.175
    k_consumption: 0.063
    k_encounter: 0.063
    k_metabolism: 0.0855
    mortality_coeff_per_yr: 0.1
    mortality_type: constant


## Run the model

In [8]:
FEISTY_driver.submit_run(
    run_config_file,
    run_forcing_file,
    run_settings_file,
)

submitting run
qsub /glade/scratch/mclong/tmp/feisty-run.f6myapex.sh
4268950.casper-pbs


0